## Importazione librerie

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Add, Activation, Dense, Lambda
from tensorflow.keras.optimizers import Adam
import scipy.stats

## Caricamento del dataset originale
Il dataset originale è composto da 4 colonne:   
- user_id: identificatore dell'utente
- date: giorno in cui è stato raccolto il dato aggregato
- data_type: intero che identifica la tipologia di dato raccolto
- data_value: valore associato al dato raccolto

I diversi valori della variabile data_type si riferiscono a dati aggregati raccolti giornalmente dal dispositivo di ogni singolo utente e identificano le tipologie di dato in base al seguente schema:
- 1: numero totale di passi effettuati
- 2: peso (kg)
- 3: BMI (kg/m^2)
- 4: pressione sanguigna sistolica (mmHg)
- 5: velocità dell'onda sfigmica arteriosa (PWV), (m/s)
- 6: PWV healthiness (1: bassa, 2: sano, 3: troppo alta)
- 7: frequenza cardiaca media (bpm)
- 8: frequenza cardiaca minima (bpm)
- 9: frequenza cardiaca massima (bpm)
- 10: durata del sonno (ore)
- 11: orario in cui l'utente si è messo a letto
- 12: orario in cui l'utente si è alzato dal letto
- 13: numero di volte in cui l'utente si è svegliato durante il sonno
- 14: durata del tempo in cui l'utente si è svegliato durante il sonno (ore)
- 15: tempo impiegato dall'utente per addormentarsi (ore)
- 16: tempo impiegato dall'utente per alzarsi dal letto (ore)
- 17: durata di sonno leggero (ore)
- 18: durata di sonno REM (ore)
- 19: durata di sonno profondo (ore)
- 20: tipo di attività
- 21: durata dell'attività (secondi)
- 22: calorie consumate durante l'attività
- 23: frequenza cardiaca media durante l'attività (bpm)
- 24: frequenza cardiaca minima durante l'attività (bpm)
- 25: frequenza cardiaca massima durante l'attività (bpm)
- 26: velocità dell'andatura dei passi (passi al minuto)
- 27: velocità dell'andatura a distanza (km all'ora)

In [2]:
raw_dataset = pd.read_csv('./data.csv', usecols=['user_id', 'date', 'data_type', 'data_value'])

raw_dataset.head()

,user_id,date,data_type,data_value
0,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,1,10131.39
1,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,2,86.80
2,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,3,28.96
3,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,7,117.62
4,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,8,117.86


## Creazione del dataset personalizzato
L'obiettivo del progetto è quello di verificare, attraverso strumenti di deep learning, l'esistenza di una correlazione fra la qualità del sonno di un utente e l'andatura dei suoi passi. Per raggiungere tale scopo è necessario costruire un nuovo dataset personalizzato contenente i soli dati utili per questa analisi, in particolare i dati riguardanti il sonno e l'andatura dei passi degli utenti.

In [3]:
feature_types = [10, 11, 12, 14, 15, 18, 19, 26]
feature_names = ["sleepduration", "bedin", "bedout", "awakeduration", "timetosleep", "remduration", "deepduration", "stepsgaitspeed"]

dataset_array = []

for i in range(len(feature_types)):
    dataset_feature = raw_dataset[raw_dataset['data_type'] == feature_types[i]].copy(deep=True)
    dataset_feature.drop(raw_dataset.columns[[2]], axis=1, inplace=True)
    dataset_feature.columns = ["user_id", "date", feature_names[i]]
    dataset_array.append(dataset_feature)

dataset = dataset_array[0]
for i in range(1, len(dataset_array)):
    dataset = dataset.merge(dataset_array[i], on=["user_id", "date"], how="outer")

# rimozione delle righe del dataset che non contengono i valori più importanti per lo studio
dataset = dataset.dropna(subset=['sleepduration', 'stepsgaitspeed'])

dataset.set_index(['user_id','date'], inplace=True)

# salvataggio del dataset in un file locale
dataset.to_csv('dataset.csv')
dataset.head()

sleepduration  bedin  \
user_id                                  date                               
1003e58667235e01b49008155604980b3900b00e 2016-08-01          11.31  22.11   
                                         2016-08-10           9.15  22.34   
                                         2016-08-15           7.87  23.93   
                                         2016-09-05           8.34  22.70   
                                         2016-09-06           8.08  23.55   

                                                     bedout  awakeduration  \
user_id                                  date                                
1003e58667235e01b49008155604980b3900b00e 2016-08-01    9.86           0.53   
                                         2016-08-10    7.66           0.28   
                                         2016-08-15    7.95           0.27   
                                         2016-09-05    7.16           0.28   
                                         2016-09-06    7.72           0.20   

                                                     timetosleep  remduration  \
user_id                                  date                                   
1003e58667235e01b49008155604980b3900b00e 2016-08-01         0.15          NaN   
                                         2016-08-10         0.02          NaN   
                                         2016-08-15         0.02          NaN   
                                         2016-09-05         0.10          NaN   
                                         2016-09-06         0.00          NaN   

                                                     deepduration  \
user_id                                  date                       
1003e58667235e01b49008155604980b3900b00e 2016-08-01          3.99   
                                         2016-08-10          3.58   
                                         2016-08-15          3.19   
                                         2016-09-05          3.11   
                                         2016-09-06          3.60   

                                                     stepsgaitspeed  
user_id                                  date                        
1003e58667235e01b49008155604980b3900b00e 2016-08-01          108.81  
                                         2016-08-10          105.02  
                                         2016-08-15           88.27  
                                         2016-09-05           84.91  
                                         2016-09-06           88.60

## Analisi del dataset personalizzato


In [4]:
dataset = pd.read_csv('./dataset.csv')

Per estrarre delle sequenze di giorni significative, è necessario trovare nei dati delle sequenze di giorni che non abbiano troppi dati mancanti. Nello studio sono state selezionate delle sequenze di giorni di lunghezza 15 in cui ogni giorno è seguito al più da un singolo giorno mancante.

In [5]:
# metodo che trasforma una data in un intero (la data zero è allineata a quella del dataset e corrisponde al giorno 2016-04-01)
def date_to_int(date):
    d1 = datetime.strptime('2016-04-01', "%Y-%m-%d")
    d2 = datetime.strptime(date, "%Y-%m-%d")
    return (d2 - d1).days

In [6]:
# trasformazione delle date in interi per semplificare il confronto
dataset['date'] = dataset['date'].apply(date_to_int)
dataset.set_index(['user_id','date'], inplace=True)
dataset.head()

sleepduration  bedin  bedout  \
user_id                                  date                                 
1003e58667235e01b49008155604980b3900b00e 122           11.31  22.11    9.86   
                                         131            9.15  22.34    7.66   
                                         136            7.87  23.93    7.95   
                                         157            8.34  22.70    7.16   
                                         158            8.08  23.55    7.72   

                                               awakeduration  timetosleep  \
user_id                                  date                               
1003e58667235e01b49008155604980b3900b00e 122            0.53         0.15   
                                         131            0.28         0.02   
                                         136            0.27         0.02   
                                         157            0.28         0.10   
                                         158            0.20         0.00   

                                               remduration  deepduration  \
user_id                                  date                              
1003e58667235e01b49008155604980b3900b00e 122           NaN          3.99   
                                         131           NaN          3.58   
                                         136           NaN          3.19   
                                         157           NaN          3.11   
                                         158           NaN          3.60   

                                               stepsgaitspeed  
user_id                                  date                  
1003e58667235e01b49008155604980b3900b00e 122           108.81  
                                         131           105.02  
                                         136            88.27  
                                         157            84.91  
                                         158            88.60

In [7]:
target_sequence_len = 15
last_user = None
last_date = None
current_sequence = []
sequences = []
count = 1

# estrazione di sequenze di lunghezza target_sequence_len che soddisfano i criteri stabiliti (ogni giorno è seguito al più da un singolo giorno mancante)
for index, row in dataset.iterrows():
    user, date = index
    if last_user != None:
        if user == last_user:
            if date == last_date + 1 or date == last_date + 2:
                current_sequence.append((index, row))
                if len(current_sequence) == target_sequence_len:
                    sequences.append(current_sequence[::])
                    current_sequence.pop(0)
            else:
                current_sequence = []
        else:
            current_sequence = []
    else:
        current_sequence.append((index, row))
    last_user = user
    last_date = date

print(f'trovate {len(sequences)} sequenze di lunghezza {target_sequence_len}')

trovate 273472 sequenze di lunghezza 15


In [8]:
# visualizzazione dei giorni considerati per alcune sequenze
for sequence in sequences[20:30]:
    test_sequence = []
    for index, row in sequence:
        test_sequence.append(index[1])
    print(test_sequence)

[344, 345, 346, 348, 349, 351, 352, 353, 354, 355, 357, 358, 359, 360, 361]
[345, 346, 348, 349, 351, 352, 353, 354, 355, 357, 358, 359, 360, 361, 362]
[346, 348, 349, 351, 352, 353, 354, 355, 357, 358, 359, 360, 361, 362, 364]
[172, 173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188]
[173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189]
[174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191]
[175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192]
[176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193]
[178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194]
[179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195]


Per poter estrarre dalle sequenze di giorni dei valori rilevanti più semplici da confrontare, è necessario addestrare un modello sull'analisi delle sequenze. In questo progetto è stato scelto come task di addestramento dei modelli il task di forecasting. Dunque, data una sequenza di 14 giorni contenente i valori di una variabile nei diversi giorni come input, il modello dovrà cercare di prevedere il valore di tale variabile al giorno 15.
Per poter soddisfare gli obiettivi del progetto è necessario addestrare due modelli diversi, uno per la previsione della qualità del sonno e l'altro per la previsione dell'andatura dei passi. È dunque necessario creare i dati di addestramento per tali modelli.

Per quanto riguarda la qualità del sonno, questa viene calcolata attraverso una versione rivisitata del Pittsburgh Sleep Quality Index (PSQI) in modo da poter includere nel calcolo dell'indice anche i dati riguardanti la durata del sonno profondo e la durata del sonno REM. Per addestrare il modello per la previsione della qualità del sonno è quindi necessario calcolare l'indice di qualità del sonno per ogni giorno a partire dai dati nelle sequenze. Per quanto riguarda invece le sequenze con i dati sull'andatura dei passi, è sufficiente raccogliere i dati giornalieri in un insieme separato.

In [9]:
def get_sleep_quality(timetosleep, sleepduration, bedin, bedout, awakeduration, remduration, deepduration):
    sleep_quality = 0

    if not pd.isna(timetosleep):
        if timetosleep > 0.25 and timetosleep <= 0.5:
            sleep_quality += 1
        elif timetosleep > 0.5 and timetosleep <= 1:
            sleep_quality += 2
        elif timetosleep > 1:
            sleep_quality += 3

    if not sleepduration:
        return 14
    
    if sleepduration > 6 and sleepduration <= 7:
        sleep_quality += 1
    elif sleepduration > 5 and sleepduration <= 6:
        sleep_quality += 2
    elif sleepduration <= 5:
        sleep_quality += 3

    if not pd.isna(bedin) and not pd.isna(bedout):
        timeinbed = bedout + 24 - bedin
        sleep_efficiency = sleepduration / timeinbed

        if sleep_efficiency > 0.75 and sleep_efficiency <= 0.85:
            sleep_quality += 1
        elif sleep_efficiency > 0.65 and sleep_efficiency <= 0.75:
            sleep_quality += 2
        elif sleep_efficiency <= 0.65:
            sleep_quality += 3

    if not pd.isna(awakeduration):
        if awakeduration > 1/3 and awakeduration <= 0.5:
            sleep_quality += 1
        elif awakeduration > 0.5 and awakeduration <= 2/3:
            sleep_quality += 2
        elif awakeduration > 2/3:
            sleep_quality += 3

    if not pd.isna(deepduration):
        percentage_deep = deepduration / sleepduration

        if percentage_deep <= 0.1:
            sleep_quality += 1

    if not pd.isna(remduration):
        percentage_rem = remduration / sleepduration

        if percentage_rem < 0.2 or percentage_rem > 0.25:
            sleep_quality += 1
    
    return sleep_quality

In [10]:
sleep_duration_sequences = []
sleep_quality_sequences = []
steps_gait_speed_sequences = []

for sequence in sequences:
    sleep_duration_sequence = []
    sleep_quality_sequence = []
    steps_gait_speed_sequence = []
    for index, row in sequence:
        sleep_duration_sequence.append(row['sleepduration'])

        sleep_quality = get_sleep_quality(row['timetosleep'], row['sleepduration'], row['bedin'], row['bedout'], row['awakeduration'], row['remduration'], row['deepduration'])
        sleep_quality_sequence.append(sleep_quality)

        steps_gait_speed_sequence.append(row['stepsgaitspeed'])
    
    sleep_duration_sequences.append(sleep_duration_sequence)
    sleep_quality_sequences.append(sleep_quality_sequence)
    steps_gait_speed_sequences.append(steps_gait_speed_sequence)

In [20]:
# creazione degli insiemi utili all'addestramento (sleep quality index)
sleep_duration_x = []
sleep_duration_y = []

for sleep_duration_sequence in sleep_duration_sequences:
    sleep_duration_x.append(sleep_duration_sequence[:14])
    sleep_duration_y.append(sleep_duration_sequence[14])

In [12]:
# creazione degli insiemi utili all'addestramento (steps gait speed)
steps_gait_x = []
steps_gait_y = []

for steps_gait_speed_sequence in steps_gait_speed_sequences:
    steps_gait_x.append(steps_gait_speed_sequence[:14])
    steps_gait_y.append(steps_gait_speed_sequence[14])

## Definizione del modello WaveNet

In [13]:
def get_wavenet(num_filters, filter_size, dilation_rates):
    # Input
    inputs = Input(shape=(target_sequence_len - 1, 1))

    # Strato di convoluzione iniziale
    conv_out = Conv1D(filters=num_filters, kernel_size=filter_size, padding='causal', dilation_rate=1)(inputs)

    # Strati convoluzionali dilatati
    for dilation_rate in dilation_rates:
        conv = Conv1D(filters=num_filters, kernel_size=filter_size, padding='causal', dilation_rate=dilation_rate)(conv_out)
        conv = Activation('relu')(conv)
        conv_out = Add()([conv_out, conv])

    # Strato di output finale
    outputs = Conv1D(filters=1, kernel_size=1, activation='linear')(conv_out)
    outputs = Lambda(lambda x: x[:, -1, :])(outputs)

    # Costruzione del modello
    model = Model(inputs=inputs, outputs=outputs)

    # Compilazione del modello
    model.compile(optimizer=Adam(), loss='mse')

    return model

In [23]:
train_set_len = (len(sleep_duration_x) * 80) // 100

sleep_duration_train_x = sleep_duration_x[:train_set_len]
sleep_duration_train_y = sleep_duration_y[:train_set_len]

sleep_duration_train_x = np.array(sleep_duration_train_x)
sleep_duration_train_y = np.array(sleep_duration_train_y)

# definizione dei parametri del modello
num_filters = 32
filter_size = 2
dilation_rates = [1, 2, 4, 8]

model_sleep_duration = get_wavenet(num_filters, filter_size, dilation_rates)

# Reshape dei dati per il modello
x = sleep_duration_train_x[..., np.newaxis]

# Addestramento del modello
history_sleep_duration = model_sleep_duration.fit(x, sleep_duration_train_y, epochs=10, batch_size=1024, validation_split=0.2)

Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 2.3495 - val_loss: 1.9176
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.9229 - val_loss: 1.9001
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.9036 - val_loss: 1.8832
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.9038 - val_loss: 1.8950
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.9009 - val_loss: 1.8822
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.8848 - val_loss: 1.8956
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.8843 - val_loss: 1.8858
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.9011 - val_loss: 1.8709
Epoch 9/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.8968 - val_loss: 1.8828
Epoch 10/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.9094 - val_loss: 1.8815


In [15]:
steps_gait_train_x = steps_gait_x[:train_set_len]
steps_gait_train_y = steps_gait_y[:train_set_len]

steps_gait_train_x = np.array(steps_gait_train_x)
steps_gait_train_y = np.array(steps_gait_train_y)

# definizione dei parametri del modello
num_filters = 32
filter_size = 2
dilation_rates = [1, 2, 4, 8]

model_steps_gait = get_wavenet(num_filters, filter_size, dilation_rates)

# Reshape dei dati per il modello
x = steps_gait_train_x[..., np.newaxis]

# Addestramento del modello
history_steps_gait = model_steps_gait.fit(x, steps_gait_train_y, epochs=10, batch_size=1024, validation_split=0.2)

Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 890.6432 - val_loss: 106.3239
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 104.8135 - val_loss: 104.3492
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 103.2693 - val_loss: 102.8691
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 102.1311 - val_loss: 102.4477
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 102.6265 - val_loss: 102.0603
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 101.7031 - val_loss: 101.8934
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 102.8254 - val_loss: 101.7454
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 101.6940 - val_loss: 101.8050
Epoch 9/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 102.8182 - val_loss: 102.1400
Epoch 10/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 100.8440 - val_loss: 101.5773


In [24]:
model_sleep_duration.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 14, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 14, 32)    │         96 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 14, 32)    │      2,080 │ conv1d_18[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 14, 32)    │          0 │ conv1d_19[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 14, 32)    │          0 │ conv1d_18[0][0],  │
│                     │                   │            │ activation_12[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_20 (Conv1D)  │ (None, 14, 32)    │      2,080 │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_13       │ (None, 14, 32)    │          0 │ conv1d_20[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, 14, 32)    │          0 │ add_12[0][0],     │
│                     │                   │            │ activation_13[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_21 (Conv1D)  │ (None, 14, 32)    │      2,080 │ add_13[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 14, 32)    │          0 │ conv1d_21[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_14 (Add)        │ (None, 14, 32)    │          0 │ add_13[0][0],     │
│                     │                   │            │ activation_14[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_22 (Conv1D)  │ (None, 14, 32)    │      2,080 │ add_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 14, 32)    │          0 │ conv1d_22[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_15 (Add)        │ (None, 14, 32)    │          0 │ add_14[0][0],     │
│                     │                   │            │ activation_15[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_23 (Conv1D)  │ (None, 14, 1)     │         33 │ add_15[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 1)         │          0 │ conv1d_23[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,349 (99.02 KB)

 Trainable params: 8,449 (33.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,900 (66.02 KB)

In [38]:
layer_name = 'conv1d_23'

intermediate_model_sleep_duration = Model(inputs=model_sleep_duration.input, outputs=model_sleep_duration.get_layer(layer_name).output)

sleep_duration_activations = intermediate_model_sleep_duration.predict(sleep_duration_train_x)

print(sleep_duration_activations.shape)
print(sleep_duration_activations[0])

6837/6837 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step
(218777, 14, 1)
[[2.994612 ]
 [4.628503 ]
 [3.7325888]
 [4.9266396]
 [5.2810273]
 [5.958565 ]
 [6.1684785]
 [6.493549 ]
 [6.0934486]
 [6.413744 ]
 [7.056337 ]
 [7.1865196]
 [7.6221666]
 [7.8130713]]


In [26]:
model_steps_gait.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 14, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 14, 32)    │         96 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 14, 32)    │      2,080 │ conv1d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 14, 32)    │          0 │ conv1d_7[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 14, 32)    │          0 │ conv1d_6[0][0],   │
│                     │                   │            │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 14, 32)    │      2,080 │ add_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 14, 32)    │          0 │ conv1d_8[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 14, 32)    │          0 │ add_4[0][0],      │
│                     │                   │            │ activation_5[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 14, 32)    │      2,080 │ add_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 14, 32)    │          0 │ conv1d_9[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 14, 32)    │          0 │ add_5[0][0],      │
│                     │                   │            │ activation_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 14, 32)    │      2,080 │ add_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 14, 32)    │          0 │ conv1d_10[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 14, 32)    │          0 │ add_6[0][0],      │
│                     │                   │            │ activation_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 14, 1)     │         33 │ add_7[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ conv1d_11[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,349 (99.02 KB)

 Trainable params: 8,449 (33.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,900 (66.02 KB)

In [40]:
layer_name = 'conv1d_11'

intermediate_model_steps_gait = Model(inputs=model_steps_gait.input, outputs=model_steps_gait.get_layer(layer_name).output)

steps_gait_activations = intermediate_model_steps_gait.predict(steps_gait_train_x)

print(steps_gait_activations.shape)
print(steps_gait_activations[0])

6837/6837 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step
(218777, 14, 1)
[[ 35.019802]
 [ 32.45654 ]
 [ 45.66244 ]
 [ 56.05827 ]
 [ 61.16846 ]
 [ 72.07842 ]
 [ 77.06309 ]
 [ 77.677864]
 [ 97.37164 ]
 [ 93.204636]
 [ 98.14143 ]
 [104.672676]
 [106.28348 ]
 [110.38056 ]]


In [29]:
sleep_quality_x = []
sleep_quality_y = []

for sleep_quality_sequence in sleep_quality_sequences:
    sleep_quality_x.append(sleep_quality_sequence[:14])
    sleep_quality_y.append(sleep_quality_sequence[14])

In [32]:
activations_train_x = []

for i in range(len(sleep_duration_activations)):
    activations_train_x.append(np.concatenate((sleep_duration_activations[i], steps_gait_activations[i])))

In [33]:
activations_train_x[0]

array([[  2.5956178],
       [  4.460799 ],
       [  3.47683  ],
       [  4.4864917],
       [  4.6182556],
       [  5.218621 ],
       [  5.864154 ],
       [  5.882618 ],
       [  5.8986607],
       [  6.244464 ],
       [  6.0390015],
       [  6.6955833],
       [  7.23788  ],
       [  7.5546403],
       [ 37.167816 ],
       [ 36.419903 ],
       [ 40.61738  ],
       [ 47.19294  ],
       [ 55.901188 ],
       [ 59.78216  ],
       [ 70.05102  ],
       [ 72.26807  ],
       [ 90.75226  ],
       [ 89.897255 ],
       [ 92.17533  ],
       [ 93.863815 ],
       [ 98.86487  ],
       [103.82161  ]], dtype=float32)

In [37]:
len(activations_train_x[0])

28

In [ ]:
activations_train_x = np.array(activations_train_x)
steps_gait_train_y = np.array(steps_gait_train_y)

# definizione dei parametri del modello
num_filters = 32
filter_size = 2
dilation_rates = [1, 2, 4, 8]

model_steps_gait = get_wavenet(num_filters, filter_size, dilation_rates)

# Reshape dei dati per il modello
x = steps_gait_train_x[..., np.newaxis]

# Addestramento del modello
history_steps_gait = model_steps_gait.fit(x, steps_gait_train_y, epochs=10, batch_size=1024, validation_split=0.2)